### Kurs ansökningsomgång kurser EDA

In [1]:
import plotly.express as px
import pandas as pd

df = pd.read_excel(
    "../data/resultat_kurser/resultat-2025-for-kurser-inom-yh.xlsx",
    sheet_name="Lista ansökningar",
    #skiprows=5,
)

df.head()

,Diarienummer,Beslut,Anordnare namn,Utbildningsnamn,Utbildningsområde,Antal beviljade platser start 2025,Antal beviljade platser start 2026,Totalt antal beviljade platser,YH-poäng,Kommun,Län,FA-region,Antal kommuner,Antal län,Antal FA-regioner
0,MYH 2025/1001,Avslag,"Härryda kommun, Yrkeshögskolan i Mölnlycke",Frihandel och ursprung - fördjupning,"Ekonomi, administration och försäljning",0,0,0,30,Härryda,Västra Götaland,Göteborg,1,1,1
1,MYH 2025/1002,Beviljad,TUC Sweden AB - Yrkeshögskola,IT- och informationssäkerhet för administrativ...,Data/IT,30,30,60,30,"Se ""Lista flera kommuner""","Se ""Lista flera kommuner""","Se ""Lista flera kommuner""",5,5,5
2,MYH 2025/1003,Beviljad,IHM Business School AB Göteborg,Digital redovisning & hållbarhetsrapportering,"Ekonomi, administration och försäljning",40,40,80,30,Göteborg,Västra Götaland,Göteborg,1,1,1
3,MYH 2025/1004,Beviljad,Lexicon Yrkeshögskola AB,Digitalisera dina utbildningar,Pedagogik och undervisning,35,35,70,50,Stockholm,Stockholm,Stockholm,1,1,1
4,MYH 2025/1005,Beviljad,Lexicon Yrkeshögskola AB,Producera digitala utbildningar,"Kultur, media och design",35,0,35,50,Stockholm,Stockholm,Stockholm,1,1,1


In [2]:
df["Län"].value_counts()

Län
Stockholm                    308
Se "Lista flera kommuner"    256
Västra Götaland              193
Skåne                         64
Örebro                        37
Västmanland                   25
Södermanland                  17
Västerbotten                  17
Kronoberg                     14
Jönköping                     14
Östergötland                  14
Västernorrland                13
Norrbotten                    10
Gävleborg                     10
Uppsala                        8
Dalarna                        7
Värmland                       6
Blekinge                       5
Kalmar                         4
Halland                        2
Jämtland                       2
Name: count, dtype: int64

In [3]:
px.bar(df["Län"].value_counts(), )

In [4]:
df.query("Län == 'Stockholm'")[
    "Utbildningsområde"
].value_counts().reset_index().rename({"count": "Ansökta utbildningar"}, axis=1)

,Utbildningsområde,Ansökta utbildningar
0,"Ekonomi, administration och försäljning",99
1,Data/IT,65
2,Samhällsbyggnad och byggteknik,41
3,Hälso- och sjukvård samt socialt arbete,23
4,Säkerhetstjänster,15
5,"Lantbruk, djurvård, trädgård, skog och fiske",11
6,Juridik,11
7,"Kultur, media och design",10
8,"Hotell, restaurang och turism",9
9,Teknik och tillverkning,8


In [5]:
# målet
# antalet sökta utbildningar per Län

def filter_per_county(df, county="Stockholm"):
    return (
        df.query("Län == @county")["Utbildningsområde"]
        .value_counts()
        .reset_index()
        .rename({"count": "Ansökta utbildningar"}, axis=1)
    )

filter_per_county(df)

,Utbildningsområde,Ansökta utbildningar
0,"Ekonomi, administration och försäljning",99
1,Data/IT,65
2,Samhällsbyggnad och byggteknik,41
3,Hälso- och sjukvård samt socialt arbete,23
4,Säkerhetstjänster,15
5,"Lantbruk, djurvård, trädgård, skog och fiske",11
6,Juridik,11
7,"Kultur, media och design",10
8,"Hotell, restaurang och turism",9
9,Teknik och tillverkning,8


In [6]:
# målet 
# antalet sökta utbildningar per Län
# antalet beviljade utbildningar per Län
# beviljandegrad 

def filter_per_county(df: pd.DataFrame, county: str = "Stockholm") -> pd.DataFrame:
    """
    Creates a summary DataFrame for a specific county showing education areas,
    total applications, approved applications, and approval rate.
    
    Parameters:
        df: DataFrame with course data
        county: Name of county to analyze (default: "Stockholm")
    
    Returns:
        DataFrame with columns: Utbildningsområde, Ansökta utbildningar, 
        Beviljade utbildningar, Beviljandegrad
    """
    # Get total applications per education area
    total_df = (
        df.query("Län == @county")["Utbildningsområde"]
        .value_counts()
        .reset_index()
        .rename({"count": "Ansökta utbildningar"}, axis=1)
    )
    
    # Get approved applications per education area
    approved_df = (
        df.query("Län == @county and Beslut == 'Beviljad'")["Utbildningsområde"]
        .value_counts()
        .reset_index()
        .rename({"count": "Beviljade utbildningar"}, axis=1)
    )
    
    # Merge total and approved
    result_df = pd.merge(
        total_df, 
        approved_df, 
        on="Utbildningsområde", 
        how="left"
    ).fillna(0)
    
    # Calculate approval rate
    result_df["Beviljandegrad"] = (
        (result_df["Beviljade utbildningar"] / result_df["Ansökta utbildningar"] * 100)
        .round(1)
    )
    
    # Sort by total applications
    result_df = result_df.sort_values("Ansökta utbildningar", ascending=True)
    
    return result_df

filter_per_county(df)

,Utbildningsområde,Ansökta utbildningar,Beviljade utbildningar,Beviljandegrad
12,Transporttjänster,1,0.0,0.0
13,Övrigt,1,0.0,0.0
10,Pedagogik och undervisning,7,6.0,85.7
11,Journalistik och information,7,5.0,71.4
9,Teknik och tillverkning,8,7.0,87.5
8,"Hotell, restaurang och turism",9,9.0,100.0
7,"Kultur, media och design",10,7.0,70.0
5,"Lantbruk, djurvård, trädgård, skog och fiske",11,1.0,9.1
6,Juridik,11,7.0,63.6
4,Säkerhetstjänster,15,9.0,60.0


### Plotly: Bar graph showing the breakdown between different education areas (Utbildningsområde) for a selected County (Län)

##### Stapeldiagrammet är uppdelat i respektive utbildningsområde och visar på antalet beviljade utbildningar i blått och antalet avslag i grått.

In [7]:
# Color constants
GRAY_1 = "#CCCCCC"
GRAY_2 = "#657072"
GRAY_3 = "#4A606C"
BLUE_1 = "#1f77b4"
BLUE_11 = "#0284c7"
GRAY_11 = "#d1d5db"
GRAY_12 = "#989898"
GRAY_13 = "#4A606C"

In [8]:
import plotly.graph_objects as go

def create_county_bar(df_summary: pd.DataFrame, 
                     county: str,
                     show_arrows: bool = True,
                     height: int = 500,
                     **options) -> go.Figure:
    """
    Creates a horizontal bar chart showing total and approved applications per education area.
    
    Parameters:
        df_summary: DataFrame with columns [Utbildningsområde, Ansökta utbildningar, 
                   Beviljade utbildningar, Beviljandegrad]
        county: Name of county for title
        show_arrows: Whether to show annotation arrows (default: True)
        height: Height of the plot in pixels (default: 500)
        **options: Additional options for customization
        
    Returns:
        Plotly Figure object
    """
    # Create figure
    fig = go.Figure()
    
    # Get categories (education areas)
    categories = df_summary["Utbildningsområde"].tolist()
    
    # Add total applications bars
    fig.add_trace(go.Bar(
        y=categories,
        x=df_summary["Ansökta utbildningar"],
        name="Totalt",
        orientation="h",
        marker_color=GRAY_11
    ))
    
    # Add approved applications bars
    fig.add_trace(go.Bar(
        y=categories,
        x=df_summary["Beviljade utbildningar"],
        name="Beviljad",
        orientation="h",
        marker_color=BLUE_11
    ))
    
    # Prepare approval rate annotations
    approval_annotations = []
    for i, row in df_summary.iterrows():
        approval_annotations.append(dict(
            x=row["Ansökta utbildningar"] + max(df_summary["Ansökta utbildningar"]) * 0.01,
            y=row["Utbildningsområde"],
            text=f"{row['Beviljandegrad']}%",
            showarrow=False,
            font=dict(color=GRAY_12, size=12),
            xanchor="left",
            yanchor="middle"
        ))
    
    # Add arrow annotations if requested
    all_annotations = approval_annotations
    if show_arrows:
        # Get values for the top category
        first_category = categories[-1]
        first_value_approved = df_summary["Beviljade utbildningar"].iloc[-1]
        first_value_total = df_summary["Ansökta utbildningar"].iloc[-1]
        
        arrow_annotations = [
            dict(
                xref="x", yref="y",
                x=first_value_approved,
                y=first_category,
                text="antal beviljade",
                showarrow=True,
                arrowhead=5,
                ax=20, ay=-70,
                font=dict(color=BLUE_11, size=13),
                arrowcolor=BLUE_11,
                arrowwidth=2,
                xanchor="auto",
                yanchor="top",
                standoff=5
            ),
            dict(
                xref="x", yref="y",
                x=first_value_total,
                y=first_category,
                text="totalt antal",
                showarrow=True,
                arrowhead=5,
                ax=0, ay=-50,
                font=dict(color=GRAY_12, size=13),
                arrowcolor=GRAY_12,
                arrowwidth=2,
                xanchor="auto",
                yanchor="top",
                standoff=5
            )
        ]
        all_annotations.extend(arrow_annotations)
    
    # Layout customization
    fig.update_layout(
        title=dict(
            text=f"Antal beviljade vs. totala ansökningar för kurser i {county}",
            x=0.5,
            y=0.95
        ),
        barmode="overlay",
        margin=dict(l=100, r=30, t=80, b=40),
        height=height,
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False,
        yaxis=dict(
            showline=True,
            linecolor=GRAY_12,
            tickfont=dict(
                color=GRAY_12,
                size=13,
                family="Arial"
            )
        ),
        xaxis=dict(
            showline=True,
            linecolor=GRAY_12,
            tickfont=dict(
                color=GRAY_12,
                size=13,
                family="Arial"
            )
        ),
        annotations=all_annotations,
        **options  # Allow additional layout customization
    )
    
    return fig

# Example usage:
df_summary = filter_per_county(df, "Stockholm")
fig = create_county_bar(df_summary, "Stockholm")
fig.show()